In [26]:
from bs4 import BeautifulSoup
import requests
import datetime as dt
from urllib.parse import urljoin
import json
import re
import os 

In [59]:
NATE_URL = 'https://pann.nate.com/talk/c20028'
#c20028 : 엔터톡 1102 ~ 1108 완료
#c20038 : 10대이야기 1102 ~ 1108 완료
#c20002 : 20대이야기 1102 ~ 1108 완료
#c20003 : 30대이야기 1102 ~ 1108 완료
#c20016 : 대한민국 이슈 1102 ~ 1108 완료
#c20050 : TV톡 1102 ~ 1108 완료
#c20001 : 사는 얘기 1102 ~ 1108 완료 
#c20008 : 사랑 고백해도 될까 1102 ~ 1108 완료 / 1015 ~ 1101 완료

NATE_HEADERS = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

In [55]:
def fetch_posts_by_date_range(start_date, end_date):
    links = []
    current_page = 1
    continue_search = True

    # 날짜 형식을 datetime 객체로 변환
    start_date = dt.datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = dt.datetime.strptime(end_date, "%Y-%m-%d").date()
    today = dt.datetime.now().date()

    while continue_search:
        url = f"{NATE_URL}?page={current_page}"
        resp = requests.get(url, headers=NATE_HEADERS)
        resp.raise_for_status()
        
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')

        # 각 게시물의 정보를 추출합니다.
        rows = soup.find('table', {'class': 'talk_list'}).find('tbody').find_all('tr')
        found_links_this_page =0
        
        for row in rows:
            time_td = row.find_all('td')[-1]  # 날짜가 포함된 td 추정
            date_str = time_td.text.strip()  # 날짜 문자열 추출

            # 날짜 형식 파싱
            if ':' in date_str:  # 오늘 게시된 글
                post_date = today
            else:  # 이전 날짜 형식 (yyyy.mm.dd)
                post_date = dt.datetime.strptime(date_str, "%Y.%m.%d").date()
            
            # 날짜 범위 내 여부 확인
            if start_date <= post_date <= end_date:
                link_tag = row.find('a', href=True)
                if link_tag and 'page=' in link_tag['href']:  # 게시글 링크 확인
                    link = 'https://pann.nate.com' + link_tag['href']
                    found_links_this_page += 1
                    links.append(link)
            elif post_date < start_date:
                continue_search = False
                break  # 더 이상 찾지 않아도 됨
        print(f"Processed page {current_page}, found {found_links_this_page} links, total links collected: {len(links)}")
        
        if continue_search:
            current_page += 1  # 다음 페이지로
        else:
            print("Search has exited: Reached earlier than start date.")

    return links

In [52]:
def get_natepann_content(link):
    resp = requests.get(link)
    resp.raise_for_status()
    resp.encoding = None
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')

    post_id = soup.find('form', {'name': 'pannFrm'}).find('input', {'name': 'pannId'}).get('value')
    community = 'NATEPANN'#soup.find('div', class_='post-navi').find('a', class_='home').get_text(strip=True)""
    category = soup.find('div', class_='post-navi').find_all('a')[1].get_text(strip=True)
    title = soup.find('h1').get_text(strip=True)
    vote_up = soup.find('div', class_='btnbox up').find('span', class_='count').get_text(strip=True)
    vote_down = soup.find('div', class_='btnbox down').find('span', class_='count').get_text(strip=True)
    view = soup.find('span', class_='count').get_text(strip=True)
    writer = soup.find('a', class_='writer').get_text(strip=True)
    date_str = soup.find('span', class_='date').get_text(strip=True)
    date = dt.datetime.strptime(date_str, '%Y.%m.%d %H:%M').strftime('%Y-%m-%d %H:%M:%S')  # 날짜 형식을 맞춰야 함
#     date = date.strftime('%Y-%m-%d %H:%M:%S')
    content = soup.find('div', {'id': 'contentArea'}).get_text(strip=True)

    comments = []
    comments_section = soup.find('div', {'id': 'commentBox'})
    if comments_section:
        # 'cmt_list' 내에서 댓글의 텍스트만 가져오기
        comment_list = comments_section.find_all('dd', class_='usertxt')
        for comment in comment_list:
            # 'span' 태그 안에 있는 댓글 텍스트를 가져옴
            comment_text = comment.find('span').get_text(strip=True)
            comments.append(comment_text)

    # 결과 출력
    data = {
        'id': 'NATE'+post_id,
        'community': community,
        'link': link,
        'category': category,
        'title': title,
        'vote_up': vote_up,
        'vote_down': vote_down,
        'view': view,
        'writer': writer,
        'date': date,
        'content': content,
        'comments': comments,
        'ncomment' : len(comments),
        'timestamp': dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    return data #title, date, writer, count, content.strip()

In [53]:
filename = "NATE_CONTENTS.json"
existing_ids = set()
all_content = []

if os.path.exists(filename):
    with open(filename, "r", encoding="utf-8") as f:
        try:
            existing_content = json.load(f)
            existing_ids = {item['id'] for item in existing_content}  # 기존 JSON 파일의 post_id 수집
            all_content.extend(existing_content)  # 기존 데이터를 리스트에 추가
        except json.JSONDecodeError:
            pass  # 파일이 비어있거나 포맷이 잘못된 경우 무시하고 진행

In [60]:
start_date = "2024-10-15"
end_date = "2024-11-01"

info = fetch_posts_by_date_range(start_date, end_date)
failed_links = []  # 에러가 발생한 링크를 저장할 리스트
total_links = len(info)

Processed page 1, found 0 links, total links collected: 0
Processed page 2, found 0 links, total links collected: 0
Processed page 3, found 0 links, total links collected: 0
Processed page 4, found 0 links, total links collected: 0
Processed page 5, found 0 links, total links collected: 0
Processed page 6, found 0 links, total links collected: 0
Processed page 7, found 0 links, total links collected: 0
Processed page 8, found 0 links, total links collected: 0
Processed page 9, found 0 links, total links collected: 0
Processed page 10, found 0 links, total links collected: 0
Processed page 11, found 0 links, total links collected: 0
Processed page 12, found 0 links, total links collected: 0
Processed page 13, found 0 links, total links collected: 0
Processed page 14, found 0 links, total links collected: 0
Processed page 15, found 0 links, total links collected: 0
Processed page 16, found 0 links, total links collected: 0
Processed page 17, found 0 links, total links collected: 0
Proces

Processed page 140, found 0 links, total links collected: 0
Processed page 141, found 0 links, total links collected: 0
Processed page 142, found 0 links, total links collected: 0
Processed page 143, found 2 links, total links collected: 2
Processed page 144, found 27 links, total links collected: 29
Processed page 145, found 26 links, total links collected: 55
Processed page 146, found 28 links, total links collected: 83
Processed page 147, found 27 links, total links collected: 110
Processed page 148, found 29 links, total links collected: 139
Processed page 149, found 27 links, total links collected: 166
Processed page 150, found 16 links, total links collected: 182
Processed page 151, found 9 links, total links collected: 191
Processed page 152, found 23 links, total links collected: 214
Processed page 153, found 26 links, total links collected: 240
Processed page 154, found 29 links, total links collected: 269
Processed page 155, found 27 links, total links collected: 296
Processe

Processed page 269, found 27 links, total links collected: 3221
Processed page 270, found 28 links, total links collected: 3249
Processed page 271, found 28 links, total links collected: 3277
Processed page 272, found 27 links, total links collected: 3304
Processed page 273, found 12 links, total links collected: 3316
Processed page 274, found 25 links, total links collected: 3341
Processed page 275, found 21 links, total links collected: 3362
Processed page 276, found 22 links, total links collected: 3384
Processed page 277, found 28 links, total links collected: 3412
Processed page 278, found 16 links, total links collected: 3428
Processed page 279, found 25 links, total links collected: 3453
Processed page 280, found 23 links, total links collected: 3476
Processed page 281, found 28 links, total links collected: 3504
Processed page 282, found 24 links, total links collected: 3528
Processed page 283, found 23 links, total links collected: 3551
Processed page 284, found 25 links, tota

Processed page 398, found 28 links, total links collected: 6466
Processed page 399, found 24 links, total links collected: 6490
Processed page 400, found 25 links, total links collected: 6515
Processed page 401, found 24 links, total links collected: 6539
Processed page 402, found 23 links, total links collected: 6562
Processed page 403, found 11 links, total links collected: 6573
Processed page 404, found 18 links, total links collected: 6591
Processed page 405, found 18 links, total links collected: 6609
Processed page 406, found 14 links, total links collected: 6623
Processed page 407, found 19 links, total links collected: 6642
Processed page 408, found 18 links, total links collected: 6660
Processed page 409, found 11 links, total links collected: 6671
Processed page 410, found 18 links, total links collected: 6689
Processed page 411, found 21 links, total links collected: 6710
Processed page 412, found 26 links, total links collected: 6736
Processed page 413, found 25 links, tota

In [57]:
# 새 데이터를 가져오고 기존 ID와 비교 후 추가
for idx, link in enumerate(info[:], start=1):
    try:
        content = get_natepann_content(link)
        if content['id'] not in existing_ids:  # 중복 확인
            all_content.append(content)
            existing_ids.add(content['id'])  # 중복 방지를 위해 추가된 ID를 기록
            print(f"[{idx}/{total_links}] Successfully processed: {link}")
        else:
            print(f"[{idx}/{total_links}] Duplicate ID, skipped: {content['id']}")
    except Exception as e:
        print(f"[{idx}/{total_links}] Error occurred while processing {link}: {e}")
        failed_links.append(link)  # 에러난 링크 저장

    if idx >= total_links:
        break
# 모든 데이터 처리 후 한 번만 저장
with open(filename, "w", encoding="utf-8") as f:
    json.dump(all_content, f, indent=4, ensure_ascii=False)
print(f"\nAll records saved to {filename} (Total records: {len(all_content)})")

# 실패한 링크 저장
if failed_links:
    with open("failed_links.json", "w", encoding="utf-8") as f:
        json.dump(failed_links, f, indent=4, ensure_ascii=False)
    print(f"Failed links saved to failed_links.json (Total failed: {len(failed_links)})")

print("json end")

[1/5879] Successfully processed: https://pann.nate.com/talk/373419782?page=161
[2/5879] Successfully processed: https://pann.nate.com/talk/373419779?page=161
[3/5879] Successfully processed: https://pann.nate.com/talk/373419776?page=161
[4/5879] Successfully processed: https://pann.nate.com/talk/373419771?page=161
[5/5879] Successfully processed: https://pann.nate.com/talk/373419764?page=161
[6/5879] Successfully processed: https://pann.nate.com/talk/373419742?page=161
[7/5879] Successfully processed: https://pann.nate.com/talk/373419738?page=162
[8/5879] Successfully processed: https://pann.nate.com/talk/373419726?page=162
[9/5879] Successfully processed: https://pann.nate.com/talk/373419709?page=162
[10/5879] Successfully processed: https://pann.nate.com/talk/373419702?page=162
[11/5879] Successfully processed: https://pann.nate.com/talk/373419694?page=162
[12/5879] Successfully processed: https://pann.nate.com/talk/373419678?page=162
[13/5879] Successfully processed: https://pann.na

[104/5879] Successfully processed: https://pann.nate.com/talk/373418584?page=165
[105/5879] Successfully processed: https://pann.nate.com/talk/373418580?page=165
[106/5879] Successfully processed: https://pann.nate.com/talk/373418565?page=165
[107/5879] Successfully processed: https://pann.nate.com/talk/373418551?page=166
[108/5879] Successfully processed: https://pann.nate.com/talk/373418543?page=166
[109/5879] Successfully processed: https://pann.nate.com/talk/373418526?page=166
[110/5879] Successfully processed: https://pann.nate.com/talk/373418510?page=166
[111/5879] Successfully processed: https://pann.nate.com/talk/373418501?page=166
[112/5879] Successfully processed: https://pann.nate.com/talk/373418495?page=166
[113/5879] Successfully processed: https://pann.nate.com/talk/373418486?page=166
[114/5879] Successfully processed: https://pann.nate.com/talk/373418480?page=166
[115/5879] Successfully processed: https://pann.nate.com/talk/373418469?page=166
[116/5879] Successfully proc

[206/5879] Successfully processed: https://pann.nate.com/talk/373417440?page=169
[207/5879] Successfully processed: https://pann.nate.com/talk/373417418?page=169
[208/5879] Successfully processed: https://pann.nate.com/talk/373417397?page=169
[209/5879] Successfully processed: https://pann.nate.com/talk/373417347?page=169
[210/5879] Successfully processed: https://pann.nate.com/talk/373417332?page=169
[211/5879] Successfully processed: https://pann.nate.com/talk/373417327?page=170
[212/5879] Successfully processed: https://pann.nate.com/talk/373417321?page=170
[213/5879] Successfully processed: https://pann.nate.com/talk/373417300?page=170
[214/5879] Successfully processed: https://pann.nate.com/talk/373417296?page=170
[215/5879] Successfully processed: https://pann.nate.com/talk/373417291?page=170
[216/5879] Successfully processed: https://pann.nate.com/talk/373417290?page=170
[217/5879] Successfully processed: https://pann.nate.com/talk/373417286?page=170
[218/5879] Successfully proc

[308/5879] Successfully processed: https://pann.nate.com/talk/373416606?page=173
[309/5879] Successfully processed: https://pann.nate.com/talk/373416589?page=173
[310/5879] Successfully processed: https://pann.nate.com/talk/373416587?page=173
[311/5879] Successfully processed: https://pann.nate.com/talk/373416572?page=173
[312/5879] Successfully processed: https://pann.nate.com/talk/373416568?page=173
[313/5879] Successfully processed: https://pann.nate.com/talk/373416567?page=173
[314/5879] Successfully processed: https://pann.nate.com/talk/373416565?page=173
[315/5879] Successfully processed: https://pann.nate.com/talk/373416564?page=173
[316/5879] Successfully processed: https://pann.nate.com/talk/373416560?page=173
[317/5879] Successfully processed: https://pann.nate.com/talk/373416557?page=173
[318/5879] Successfully processed: https://pann.nate.com/talk/373416554?page=173
[319/5879] Successfully processed: https://pann.nate.com/talk/373416551?page=173
[320/5879] Successfully proc

[410/5879] Successfully processed: https://pann.nate.com/talk/373415974?page=177
[411/5879] Successfully processed: https://pann.nate.com/talk/373415972?page=177
[412/5879] Successfully processed: https://pann.nate.com/talk/373415962?page=177
[413/5879] Successfully processed: https://pann.nate.com/talk/373415938?page=177
[414/5879] Successfully processed: https://pann.nate.com/talk/373415933?page=177
[415/5879] Successfully processed: https://pann.nate.com/talk/373415924?page=177
[416/5879] Successfully processed: https://pann.nate.com/talk/373415914?page=177
[417/5879] Successfully processed: https://pann.nate.com/talk/373415900?page=177
[418/5879] Successfully processed: https://pann.nate.com/talk/373415884?page=177
[419/5879] Successfully processed: https://pann.nate.com/talk/373415870?page=177
[420/5879] Successfully processed: https://pann.nate.com/talk/373415863?page=177
[421/5879] Successfully processed: https://pann.nate.com/talk/373415861?page=177
[422/5879] Successfully proc

...

[5423/5879] Successfully processed: https://pann.nate.com/talk/373343006?page=361
[5424/5879] Successfully processed: https://pann.nate.com/talk/373343002?page=361
[5425/5879] Successfully processed: https://pann.nate.com/talk/373343001?page=361
[5426/5879] Successfully processed: https://pann.nate.com/talk/373342984?page=361
[5427/5879] Successfully processed: https://pann.nate.com/talk/373342978?page=361
[5428/5879] Successfully processed: https://pann.nate.com/talk/373342953?page=362
[5429/5879] Successfully processed: https://pann.nate.com/talk/373342932?page=362
[5430/5879] Successfully processed: https://pann.nate.com/talk/373342931?page=362
[5431/5879] Successfully processed: https://pann.nate.com/talk/373342914?page=362
[5432/5879] Successfully processed: https://pann.nate.com/talk/373342856?page=362
[5433/5879] Successfully processed: https://pann.nate.com/talk/373342810?page=362
[5434/5879] Successfully processed: https://pann.nate.com/talk/373342743?page=362
[5435/5879] Succ

[5523/5879] Successfully processed: https://pann.nate.com/talk/373340671?page=366
[5524/5879] Successfully processed: https://pann.nate.com/talk/373340670?page=366
[5525/5879] Successfully processed: https://pann.nate.com/talk/373340669?page=366
[5526/5879] Successfully processed: https://pann.nate.com/talk/373340668?page=366
[5527/5879] Successfully processed: https://pann.nate.com/talk/373340577?page=366
[5528/5879] Successfully processed: https://pann.nate.com/talk/373340530?page=366
[5529/5879] Successfully processed: https://pann.nate.com/talk/373340507?page=366
[5530/5879] Successfully processed: https://pann.nate.com/talk/373340487?page=366
[5531/5879] Successfully processed: https://pann.nate.com/talk/373340421?page=366
[5532/5879] Successfully processed: https://pann.nate.com/talk/373340398?page=366
[5533/5879] Successfully processed: https://pann.nate.com/talk/373340369?page=366
[5534/5879] Successfully processed: https://pann.nate.com/talk/373340360?page=366
[5535/5879] Succ

[5623/5879] Successfully processed: https://pann.nate.com/talk/373338770?page=369
[5624/5879] Successfully processed: https://pann.nate.com/talk/373338764?page=370
[5625/5879] Successfully processed: https://pann.nate.com/talk/373338760?page=370
[5626/5879] Successfully processed: https://pann.nate.com/talk/373338757?page=370
[5627/5879] Successfully processed: https://pann.nate.com/talk/373338754?page=370
[5628/5879] Successfully processed: https://pann.nate.com/talk/373338749?page=370
[5629/5879] Successfully processed: https://pann.nate.com/talk/373338741?page=370
[5630/5879] Successfully processed: https://pann.nate.com/talk/373338732?page=370
[5631/5879] Successfully processed: https://pann.nate.com/talk/373338723?page=370
[5632/5879] Successfully processed: https://pann.nate.com/talk/373338711?page=370
[5633/5879] Successfully processed: https://pann.nate.com/talk/373338709?page=370
[5634/5879] Successfully processed: https://pann.nate.com/talk/373338700?page=370
[5635/5879] Succ

[5723/5879] Successfully processed: https://pann.nate.com/talk/373337319?page=373
[5724/5879] Successfully processed: https://pann.nate.com/talk/373337276?page=373
[5725/5879] Successfully processed: https://pann.nate.com/talk/373337264?page=373
[5726/5879] Successfully processed: https://pann.nate.com/talk/373337250?page=373
[5727/5879] Successfully processed: https://pann.nate.com/talk/373337242?page=373
[5728/5879] Successfully processed: https://pann.nate.com/talk/373337222?page=373
[5729/5879] Successfully processed: https://pann.nate.com/talk/373337216?page=373
[5730/5879] Successfully processed: https://pann.nate.com/talk/373337207?page=373
[5731/5879] Successfully processed: https://pann.nate.com/talk/373337192?page=373
[5732/5879] Successfully processed: https://pann.nate.com/talk/373337191?page=373
[5733/5879] Successfully processed: https://pann.nate.com/talk/373337161?page=374
[5734/5879] Successfully processed: https://pann.nate.com/talk/373337110?page=374
[5735/5879] Succ

[5823/5879] Successfully processed: https://pann.nate.com/talk/373335211?page=377
[5824/5879] Successfully processed: https://pann.nate.com/talk/373335187?page=377
[5825/5879] Successfully processed: https://pann.nate.com/talk/373335183?page=378
[5826/5879] Successfully processed: https://pann.nate.com/talk/373335170?page=378
[5827/5879] Successfully processed: https://pann.nate.com/talk/373335142?page=378
[5828/5879] Successfully processed: https://pann.nate.com/talk/373335133?page=378
[5829/5879] Successfully processed: https://pann.nate.com/talk/373335131?page=378
[5830/5879] Successfully processed: https://pann.nate.com/talk/373335130?page=378
[5831/5879] Successfully processed: https://pann.nate.com/talk/373335082?page=378
[5832/5879] Successfully processed: https://pann.nate.com/talk/373335061?page=378
[5833/5879] Successfully processed: https://pann.nate.com/talk/373335048?page=378
[5834/5879] Successfully processed: https://pann.nate.com/talk/373335044?page=378
[5835/5879] Succ